In [30]:
import datasets

def load_articles(n=5):
    dataset = datasets.load_dataset("sedthh/gutenberg_english", 'default', split='train', streaming='True')
    data = dataset.take(n)
    return [d['TEXT'] for d in data]

articles = load_articles()
print(articles[0])
    

/Library/Python/3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The United States Bill of Rights.
The Ten Original Amendments to the Constitution of the United States
Passed by Congress September 25, 1789
Ratified December 15, 1791


I
Congress shall make no law respecting an establishment of religion, or

prohibiting the free exercise thereof; or abridging the freedom of speech, or of

the press, or the right of the people peaceably to assemble, and to petition the

Government for a redress of grievances.
II
A well-regulated militia, being necessary to the security of a free State,

the right of the people to keep and bear arms, shall not be

infringed.
III
No soldier shall, in time of peace be quartered in any house, without the

consent of the owner, nor in time of war, but in a manner to be prescribed by

law.
IV
The right of the people to be secure in their persons, houses, papers, and

effects, against unreasonable searches and seizures, shall not be violated, and

no Warrants shall issue, but upon probable cause, supported by oath or

affirm

In [31]:
%pip uninstall langchain openai -y
%pip install langchain-openai
%pip install python-dotenv
%pip install langchain-community

from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

# Load environment variables from a .env file
load_dotenv()

# Get your OpenAI API key from the environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

Found existing installation: langchain 0.2.7
Uninstalling langchain-0.2.7:
  Successfully uninstalled langchain-0.2.7
Found existing installation: openai 1.35.13
Uninstalling openai-1.35.13:
  Successfully uninstalled openai-1.35.13
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-1.35.13-py3-none-any.whl.metadata (21 kB)
Using cached openai-1.35.13-py3-none-any.whl (328 kB)
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain-0.2.7-py3-none-any.whl.metadata (6.9 kB)
Using cached langchain-0.2.7-py3-none-any.whl (983 kB)
Note: you may need to restart the kernel to use updated packages.


In [32]:
# Create an instance of the OpenAIEmbeddings class
from langchain_text_splitters import RecursiveCharacterTextSplitter


splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = splitter.create_documents(articles)
document_chunks = splitter.split_documents(documents)
document_chunks[0]

Document(page_content='The United States Bill of Rights.\r\nThe Ten Original Amendments to the Constitution of the United States\r\nPassed by Congress September 25, 1789\r\nRatified December 15, 1791\r\n\r\n\r\nI\r\nCongress shall make no law respecting an establishment of religion, or\r\n\r\nprohibiting the free exercise thereof; or abridging the freedom of speech, or of\r\n\r\nthe press, or the right of the people peaceably to assemble, and to petition the\r\n\r\nGovernment for a redress of grievances.\r\nII\r\nA well-regulated militia, being necessary to the security of a free State,\r\n\r\nthe right of the people to keep and bear arms, shall not be\r\n\r\ninfringed.\r\nIII\r\nNo soldier shall, in time of peace be quartered in any house, without the\r\n\r\nconsent of the owner, nor in time of war, but in a manner to be prescribed by\r\n\r\nlaw.\r\nIV\r\nThe right of the people to be secure in their persons, houses, papers, and\r\n\r\neffects, against unreasonable searches and seizur

In [33]:
%pip install "cassio>=0.1.0"

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [34]:
%pip install --upgrade pip
%pip install cassandra-driver

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [35]:
import os
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Directly using the file paths instead of environment variables
secure_connect_bundle_path = "/Users/wanderson/Desktop/development/usa-constitution/secure-connect-text-qa-database.zip"
token_path = "/Users/wanderson/Desktop/development/usa-constitution/token-astradb.txt"

# Read the token from the file
with open(token_path, 'r') as token_file:
    token = token_file.read().strip()

# Set up the Cluster connection
session = Cluster(
    cloud={"secure_connect_bundle": secure_connect_bundle_path},
    auth_provider=PlainTextAuthProvider("token", token)
).connect()


In [36]:
print(session.session_id)

564f87ec-e575-405d-b5c3-964a57b416e4


In [37]:
from langchain.vectorstores import Cassandra

astra_vectorstore = Cassandra.from_documents(
    documents=document_chunks, 
    embedding=embeddings,
    session=session,
    keyspace="text_qa_keyspace",
    table_name='text_qa_vectors'
    )

In [39]:
articles[0]

'The United States Bill of Rights.\r\nThe Ten Original Amendments to the Constitution of the United States\r\nPassed by Congress September 25, 1789\r\nRatified December 15, 1791\r\n\r\n\r\nI\r\nCongress shall make no law respecting an establishment of religion, or\r\n\r\nprohibiting the free exercise thereof; or abridging the freedom of speech, or of\r\n\r\nthe press, or the right of the people peaceably to assemble, and to petition the\r\n\r\nGovernment for a redress of grievances.\r\nII\r\nA well-regulated militia, being necessary to the security of a free State,\r\n\r\nthe right of the people to keep and bear arms, shall not be\r\n\r\ninfringed.\r\nIII\r\nNo soldier shall, in time of peace be quartered in any house, without the\r\n\r\nconsent of the owner, nor in time of war, but in a manner to be prescribed by\r\n\r\nlaw.\r\nIV\r\nThe right of the people to be secure in their persons, houses, papers, and\r\n\r\neffects, against unreasonable searches and seizures, shall not be viola

In [44]:
query = "The Constitution of the United States?"
astra_vectorstore.similarity_search(query, k=100)

[Document(page_content='THE CONSTITUTION OF THE UNITED STATES OF AMERICA, 1787\r\nWe the people of the United States, in Order to form a more perfect Union,\r\n\r\nestablish Justice, insure domestic Tranquility, provide for the common defence,\r\n\r\npromote the general Welfare, and secure the Blessings of Liberty to ourselves\r\n\r\nand our Posterity, do ordain and establish this Constitution for the\r\n\r\nUnited States of America.\r\nArticle I\r\nSection 1.  All legislative Powers herein granted shall be vested in a\r\n\r\nCongress of the United States, which shall consist of a Senate and\r\n\r\nHouse of Representatives.\r\nSection 2.  The House of Representatives shall be composed of Members\r\n\r\nchosen every second Year by the People of the several States,\r\n\r\nand the electors in each State shall have the qualifications requisite\r\n\r\nfor electors of the most numerous branch of the State legislature.\r\nNo Person shall be a Representative who shall not have attained to the'

# Retrieval Augmented Generation

In [47]:
retriever = astra_vectorstore.as_retriever(search_kwargs={'k': 100})

In [50]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo')

qa_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type='stuff',
    
)

In [51]:
query = "The Constitution of the United States?"
qa_retriever.run(query)

'The Constitution of the United States is the supreme law of the United States of America. It was created in 1787 and outlines the framework of the federal government, its powers, and the rights of the citizens. It consists of a Preamble, seven Articles, and a series of Amendments, including the Bill of Rights.'